<a href="https://colab.research.google.com/github/Fayjul/AI_assingment/blob/main/AI_Assignment_4_Naive_Bais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv('/content/heart.csv') #Read the dataset
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [4]:
data.drop(["age", "trestbps", "chol", "thalach", "oldpeak", "slope"],axis = 1 ,inplace=True) #drop irrelevant columns
data.head()

,sex,cp,fbs,restecg,exang,ca,thal,target
0,1,0,0,1,0,2,3,0
1,1,0,1,0,1,0,3,0
2,1,0,0,1,1,0,3,0
3,1,0,0,1,0,1,3,0
4,0,0,1,1,0,3,2,0


In [5]:
X = data[data.keys()[:-1]]
y = data[data.keys()[-1]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)#test-train split
data_train = pd.concat([X_train, y_train],axis = 1) #concat back
data_test = pd.concat([X_test, y_test],axis = 1)

In [6]:
#Calcualting probabilites for inputs independantly
def get_probabilities_for_inputs(n, column_name, data_frame):
    
    temp = data_frame[column_name] #isolate targetted column
    temp = temp.value_counts() #get counts of occurences of each input variable
    
    return (temp/n) #return probiblity of occurence by dividing with total no. of data points

In [8]:
#calculating conditional probability
def get_conditional_probabilities(data_frame, n,target, given):
    
    focused_data = data[[target, given]] #isolate target column an dfocus input column
    targets_unique = data[target].unique()#list of unique outputs in data
    inputs_unique = data[given].unique()
    
    groups = focused_data.groupby(by = [given, target]).size().reset_index()
    groups[0] = groups[0]/ n
    
    
    for targets in targets_unique:
        current_target_length = len(focused_data[focused_data[target] == targets])
        groups[0] = np.where(groups[target] == targets, groups[0].div(current_target_length),groups[0])
    
    return groups

In [9]:
def calculate_probabilities(data):
    #splititng input data
    x = data[data.keys()[:-1]]
    y = data[data.keys()[-1]]
    target = y.name
    
    #get length of dataframe
    n = len(data)
    
    #get probabilities for each individual input and output
    f_in = lambda lst: get_probabilities_for_inputs(n, lst, x)
    input_probablities = list(map(f_in,x.keys()))
    
    output_probabilities = get_probabilities_for_inputs(n ,target, y.to_frame())
    
    #get conditional probabilities for every input against every output
    f1 = lambda lst: get_conditional_probabilities(data, n, target,lst)
    conditional_probabilities = list(map(f1, data.keys()[:-1]))
    
    return input_probablities, output_probabilities, conditional_probabilities

In [11]:
def naive_bayes_calculator(target_values, input_values, in_prob, out_prob, cond_prob):
    
    target_values.sort()#sort the target values to assure ascending order
    classes = [] #initialise empty probabilites list
    
    for target_value in target_values:
        num = 1 #initilaise numerator
        den = 1 #initialise denominator
#calculate denominator according to the formula
        for i,x in enumerate(input_values):
            den *= in_prob[i][x]
#calculate numerator according to the formula    
        for i, x_1 in enumerate(input_values):
            temp_df = cond_prob[i]
            num *= temp_df[(temp_df.iloc[:,0] == x_1) & (temp_df.iloc[:,1] == target_value)][0].values[0]
        num *= out_prob[target_value]
        final_probability = (num/den) #final conditional probability value
        
        classes.append(final_probability) #append probability for current class in a list
        
    return (classes.index(max(classes)), classes)

In [12]:
in_prob, out_prob, cond_prob = calculate_probabilities(data_train)#use training data for the initial calculations

In [13]:
#testing with dummy data
naive_bayes_calculator([1,0], [1,1,0,2,1,3,3],in_prob,out_prob,cond_prob)

(0, [2.958612139554826e-21, 1.9627596445304052e-23])

In [14]:
def naive_bayes_predictor(test_data, outputs, in_prob, out_prob, cond_prob):
    
    final_predictions = [] #initialise empty list to store test predictions
    
    for row in test_data:
        #get prediction for current data
        predicted_class, probabilities = naive_bayes_calculator(outputs, row, in_prob, out_prob, cond_prob)
        #append to list
        final_predictions.append(predicted_class)
        
        
    return final_predictions

In [15]:
test_data_as_list = X_test.values.tolist()
unique_targets = y_test.unique().tolist()
predicted_y = naive_bayes_predictor(test_data_as_list,unique_targets,in_prob,out_prob,cond_prob)
print("Accuracy:", (np.count_nonzero(y_test == predicted_y)/len(y_test)) *100)

Accuracy: 86.40776699029125
